# Notebook 2: Energy Calibration

## Introduction

### Why Energy Calibration?

Scintillation detectors produce signals proportional to deposited energy, but the raw ADC values need to be converted to physical energy units (keV). Energy calibration establishes this conversion using known gamma-ray sources.

### Calibration Approaches

1. **Linear calibration**: $E_{\text{keV}} = a \cdot \text{ADC} + b$
   - Simple, fast, adequate for narrow energy ranges
   - Assumes linear detector response

2. **Polynomial calibration**: $E_{\text{keV}} = a \cdot \text{ADC}^2 + b \cdot \text{ADC} + c$
   - Accounts for non-linearity
   - Better for wide energy ranges (keV to MeV)

3. **Spline calibration**: Piecewise interpolation
   - Most flexible
   - Requires many calibration points

### Common Calibration Sources

| Isotope | Energy (keV) | Application |
|---------|--------------|-------------|
| Co-60 | 1173.2, 1332.5 | High energy |
| Cs-137 | 661.7 | Mid energy, reference |
| Na-22 | 511.0, 1274.5 | Positron annihilation |
| Am-241 | 59.5 | Low energy |
| Ba-133 | 81.0, 276.4, 302.9, 356.0, 383.8 | Multi-point |

### Compton Edge Calibration

For organic scintillators (poor energy resolution), we often use the **Compton edge** instead of full-energy peaks:

$$E_{\text{edge}} = E_\gamma \cdot \frac{2E_\gamma}{511 + 2E_\gamma}$$

Example: Cs-137 (662 keV) → Compton edge at 477 keV

### Learning Objectives

1. Generate synthetic gamma spectra with known peaks
2. Locate Compton edges and photopeaks
3. Perform linear, polynomial, and spline calibrations
4. Validate calibration quality
5. Apply energy-dependent corrections

## Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, optimize, interpolate
from scipy.stats import norm

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

print("✓ Libraries imported")

## 1. Generate Synthetic Gamma Spectrum

We'll simulate a spectrum from Cs-137 with both Compton continuum and photopeak.

In [ ]:
def compton_edge_energy(gamma_energy_kev):
    """
    Calculate Compton edge energy
    
    E_edge = E_gamma * 2*E_gamma / (511 + 2*E_gamma)
    """
    return gamma_energy_kev * (2 * gamma_energy_kev) / (511 + 2 * gamma_energy_kev)


def generate_gamma_spectrum(isotope='Cs-137', n_events=50000, 
                           energy_to_adc=2.0, resolution_percent=8.0):
    """
    Generate synthetic gamma spectrum
    
    Parameters:
    -----------
    isotope : str
        'Cs-137', 'Co-60', 'Na-22', or 'Ba-133'
    n_events : int
        Number of events to simulate
    energy_to_adc : float
        Conversion factor (ADC counts per keV) - UNKNOWN in real data!
    resolution_percent : float
        Energy resolution (FWHM%) at 662 keV
    
    Returns:
    --------
    adc_values : array
        Simulated ADC spectrum
    true_energies_kev : dict
        True photopeak energies for validation
    """
    # Define isotope gamma lines
    isotope_energies = {
        'Cs-137': [661.7],
        'Co-60': [1173.2, 1332.5],
        'Na-22': [511.0, 1274.5],
        'Ba-133': [81.0, 276.4, 302.9, 356.0, 383.8]
    }
    
    if isotope not in isotope_energies:
        raise ValueError(f"Unknown isotope: {isotope}")
    
    energies_kev = isotope_energies[isotope]
    
    adc_values = []
    
    for energy_kev in energies_kev:
        # Calculate Compton edge
        edge_kev = compton_edge_energy(energy_kev)
        
        # Number of events for this peak
        n_peak = n_events // len(energies_kev)
        
        # Compton continuum (flat from 0 to edge, then falling)
        n_compton = int(n_peak * 0.7)  # 70% Compton scatter
        compton_energies = np.random.uniform(0, edge_kev, n_compton)
        
        # Photopeak (Gaussian)
        n_photopeak = int(n_peak * 0.3)  # 30% full energy
        sigma_kev = (resolution_percent / 100) * energy_kev / 2.355
        photopeak_energies = np.random.normal(energy_kev, sigma_kev, n_photopeak)
        
        # Combine
        all_energies = np.concatenate([compton_energies, photopeak_energies])
        
        # Convert keV → ADC (this is what we're trying to calibrate!)
        adc = all_energies * energy_to_adc
        adc_values.extend(adc)
    
    adc_values = np.array(adc_values)
    
    # Add ADC noise
    adc_values += np.random.normal(0, 5, len(adc_values))
    adc_values = adc_values[adc_values > 0]  # Remove negative values
    
    return adc_values, energies_kev


# Generate spectrum
adc_spectrum, true_energies = generate_gamma_spectrum('Cs-137', n_events=50000)

print(f"✓ Generated spectrum with {len(adc_spectrum)} events")
print(f"  True gamma energy: {true_energies[0]} keV")
print(f"  Compton edge: {compton_edge_energy(true_energies[0]):.1f} keV")
print(f"  ADC range: {adc_spectrum.min():.0f} - {adc_spectrum.max():.0f}")

## 2. Visualize Uncalibrated Spectrum

In [ ]:
# Plot uncalibrated spectrum
fig, ax = plt.subplots(figsize=(12, 6))

counts, bins, _ = ax.hist(adc_spectrum, bins=300, histtype='step', 
                          linewidth=2, color='blue', label='Uncalibrated spectrum')
bin_centers = (bins[:-1] + bins[1:]) / 2

ax.set_xlabel('ADC Channel (uncalibrated)', fontsize=13, fontweight='bold')
ax.set_ylabel('Counts', fontsize=13, fontweight='bold')
ax.set_title('Cs-137 Gamma Spectrum (ADC units)', fontsize=15, fontweight='bold')
ax.set_yscale('log')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11)

plt.tight_layout()
plt.show()

print("✓ Uncalibrated spectrum plotted")
print("\nGoal: Convert ADC channels → keV energy")

## 3. Find Compton Edge

For organic scintillators, we use the Compton edge for calibration.

In [ ]:
def find_compton_edge(spectrum_counts, bin_centers, expected_edge_kev=None, 
                     search_width_percent=20):
    """
    Locate Compton edge in spectrum
    
    Method: Find the edge as the 50% point after the maximum
    
    Parameters:
    -----------
    spectrum_counts : array
        Histogram counts
    bin_centers : array
        Bin center values (ADC or keV)
    expected_edge_kev : float, optional
        Expected edge energy (for focused search)
    search_width_percent : float
        Search range as % of spectrum
    
    Returns:
    --------
    edge_position : float
        Compton edge location (in bin units)
    """
    # Smooth spectrum for edge detection
    from scipy.ndimage import gaussian_filter1d
    smoothed = gaussian_filter1d(spectrum_counts, sigma=3)
    
    # Search in upper portion of spectrum
    search_start = int(len(bin_centers) * 0.5)
    search_end = int(len(bin_centers) * 0.9)
    
    search_spectrum = smoothed[search_start:search_end]
    search_bins = bin_centers[search_start:search_end]
    
    # Find maximum in search region
    max_idx = np.argmax(search_spectrum)
    max_counts = search_spectrum[max_idx]
    
    # Edge = 50% point after maximum
    threshold = 0.5 * max_counts
    
    # Find rightmost point above threshold
    above_thresh = search_spectrum[max_idx:] > threshold
    if above_thresh.any():
        edge_idx = max_idx + np.where(above_thresh)[0][-1]
        edge_position = search_bins[edge_idx]
    else:
        edge_position = search_bins[max_idx]
    
    return edge_position


# Find Compton edge in ADC spectrum
edge_adc = find_compton_edge(counts, bin_centers)
edge_kev_true = compton_edge_energy(true_energies[0])

print(f"✓ Compton edge located")
print(f"  Edge position (ADC): {edge_adc:.1f}")
print(f"  True edge energy: {edge_kev_true:.1f} keV")
print(f"  Implied conversion: {edge_kev_true/edge_adc:.3f} keV/ADC")

## 4. Find Photopeak (for inorganic scintillators)

For detectors with good energy resolution (NaI, HPGe), we fit the photopeak directly.

In [ ]:
def find_photopeak(spectrum_counts, bin_centers, 
                  prominence_factor=100, fit_width_bins=30):
    """
    Find and fit photopeak with Gaussian
    
    Parameters:
    -----------
    spectrum_counts : array
        Histogram counts
    bin_centers : array
        Bin center values
    prominence_factor : float
        Minimum peak prominence
    fit_width_bins : int
        Number of bins to include in fit
    
    Returns:
    --------
    peak_centroid : float
        Fitted peak center
    peak_fwhm : float
        Fitted peak FWHM
    """
    # Find peaks
    peaks, properties = signal.find_peaks(spectrum_counts, 
                                         prominence=prominence_factor,
                                         distance=20)
    
    if len(peaks) == 0:
        print("Warning: No peaks found")
        return None, None
    
    # Take highest peak
    peak_idx = peaks[np.argmax(spectrum_counts[peaks])]
    peak_adc = bin_centers[peak_idx]
    
    # Extract region around peak
    fit_start = max(0, peak_idx - fit_width_bins)
    fit_end = min(len(bin_centers), peak_idx + fit_width_bins)
    
    x_fit = bin_centers[fit_start:fit_end]
    y_fit = spectrum_counts[fit_start:fit_end]
    
    # Gaussian fit
    def gaussian(x, amp, mu, sigma, bg):
        return amp * np.exp(-0.5 * ((x - mu) / sigma)**2) + bg
    
    try:
        p0 = [y_fit.max(), peak_adc, 10, y_fit.min()]
        popt, _ = optimize.curve_fit(gaussian, x_fit, y_fit, p0=p0)
        
        amp, mu, sigma, bg = popt
        fwhm = 2.355 * abs(sigma)
        
        return mu, fwhm
    
    except:
        print("Warning: Peak fit failed")
        return peak_adc, None


# Find photopeak
peak_adc, peak_fwhm = find_photopeak(counts, bin_centers)

if peak_adc:
    print(f"✓ Photopeak located")
    print(f"  Peak position (ADC): {peak_adc:.1f}")
    print(f"  Peak FWHM (ADC): {peak_fwhm:.1f}")
    print(f"  True peak energy: {true_energies[0]} keV")
    print(f"  Implied conversion: {true_energies[0]/peak_adc:.3f} keV/ADC")

## 5. Linear Calibration

Simplest approach: $E = a \cdot \text{ADC} + b$

In [ ]:
def linear_calibration(calibration_points):
    """
    Perform linear energy calibration
    
    Parameters:
    -----------
    calibration_points : list of tuples
        [(adc1, kev1), (adc2, kev2), ...]
    
    Returns:
    --------
    cal_func : function
        Calibration function: ADC → keV
    params : array
        [slope, intercept]
    """
    adc_vals = np.array([p[0] for p in calibration_points])
    kev_vals = np.array([p[1] for p in calibration_points])
    
    # Linear fit: E = a*ADC + b
    params = np.polyfit(adc_vals, kev_vals, deg=1)
    cal_func = np.poly1d(params)
    
    # Calculate fit quality
    predicted = cal_func(adc_vals)
    residuals = kev_vals - predicted
    rms_error = np.sqrt(np.mean(residuals**2))
    
    print("Linear calibration:")
    print(f"  E[keV] = {params[0]:.4f} * ADC + {params[1]:.2f}")
    print(f"  RMS error: {rms_error:.2f} keV")
    
    return cal_func, params, rms_error


# Perform calibration using Compton edge and photopeak
calibration_points = [
    (edge_adc, edge_kev_true),  # Compton edge
    (peak_adc, true_energies[0])  # Photopeak
]

cal_func_linear, params_linear, rms_linear = linear_calibration(calibration_points)

print(f"\n✓ Linear calibration complete")

## 6. Apply Calibration and Validate

In [ ]:
# Apply calibration
energy_kev_calibrated = cal_func_linear(adc_spectrum)

# Plot calibrated spectrum
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Uncalibrated
ax = axes[0]
ax.hist(adc_spectrum, bins=300, histtype='step', linewidth=2, color='blue')
ax.axvline(edge_adc, color='red', linestyle='--', linewidth=2, 
           label=f'Compton edge ({edge_adc:.0f} ADC)')
ax.axvline(peak_adc, color='green', linestyle='--', linewidth=2, 
           label=f'Photopeak ({peak_adc:.0f} ADC)')
ax.set_xlabel('ADC Channel', fontsize=12, fontweight='bold')
ax.set_ylabel('Counts', fontsize=12, fontweight='bold')
ax.set_title('Before Calibration', fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Calibrated
ax = axes[1]
counts_cal, bins_cal, _ = ax.hist(energy_kev_calibrated, bins=300, 
                                  histtype='step', linewidth=2, color='blue')
ax.axvline(edge_kev_true, color='red', linestyle='--', linewidth=2, 
           label=f'Compton edge ({edge_kev_true:.0f} keV)')
ax.axvline(true_energies[0], color='green', linestyle='--', linewidth=2, 
           label=f'Photopeak ({true_energies[0]:.0f} keV)')
ax.set_xlabel('Energy (keV)', fontsize=12, fontweight='bold')
ax.set_ylabel('Counts', fontsize=12, fontweight='bold')
ax.set_title('After Linear Calibration', fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Calibration applied and validated")
print(f"\nCalibrated spectrum range: {energy_kev_calibrated.min():.0f} - {energy_kev_calibrated.max():.0f} keV")

## 7. Multi-Source Calibration (Polynomial)

For better accuracy across wide energy ranges, use multiple sources.

In [ ]:
# Simulate multi-source calibration data
# Generate spectra from different sources

sources = {
    'Na-22': [511.0, 1274.5],
    'Cs-137': [661.7],
    'Co-60': [1173.2, 1332.5]
}

# Simulate with slight non-linearity
def adc_to_kev_nonlinear(kev):
    """Simulate detector response with slight non-linearity"""
    return 2.0 * kev + 0.0001 * kev**2  # Slight quadratic term

calibration_points_multi = []

for source, energies in sources.items():
    for energy_kev in energies:
        adc_center = adc_to_kev_nonlinear(energy_kev)
        # Add measurement uncertainty
        adc_measured = adc_center + np.random.normal(0, 5)
        calibration_points_multi.append((adc_measured, energy_kev))

# Sort by ADC
calibration_points_multi = sorted(calibration_points_multi, key=lambda x: x[0])

print("Multi-source calibration points:")
for adc, kev in calibration_points_multi:
    print(f"  ADC: {adc:7.1f} → {kev:7.1f} keV")

## 8. Polynomial Calibration

In [ ]:
def polynomial_calibration(calibration_points, degree=2):
    """
    Polynomial energy calibration
    
    E = a*ADC^n + b*ADC^(n-1) + ... + c
    """
    adc_vals = np.array([p[0] for p in calibration_points])
    kev_vals = np.array([p[1] for p in calibration_points])
    
    # Polynomial fit
    params = np.polyfit(adc_vals, kev_vals, deg=degree)
    cal_func = np.poly1d(params)
    
    # Calculate fit quality
    predicted = cal_func(adc_vals)
    residuals = kev_vals - predicted
    rms_error = np.sqrt(np.mean(residuals**2))
    
    print(f"Polynomial calibration (degree {degree}):")
    print(f"  Coefficients: {params}")
    print(f"  RMS error: {rms_error:.2f} keV")
    
    return cal_func, params, rms_error


# Compare linear vs polynomial
cal_func_poly, params_poly, rms_poly = polynomial_calibration(
    calibration_points_multi, degree=2
)
cal_func_linear_multi, params_linear_multi, rms_linear_multi = linear_calibration(
    calibration_points_multi
)

print(f"\n✓ Polynomial calibration complete")
print(f"\nComparison:")
print(f"  Linear RMS error:     {rms_linear_multi:.3f} keV")
print(f"  Polynomial RMS error: {rms_poly:.3f} keV")
print(f"  Improvement: {(1 - rms_poly/rms_linear_multi)*100:.1f}%")

## 9. Spline Calibration

Most flexible approach using cubic splines.

In [ ]:
def spline_calibration(calibration_points, smoothing=0):
    """
    Spline interpolation calibration
    
    Parameters:
    -----------
    calibration_points : list
        Calibration data points
    smoothing : float
        Smoothing parameter (0 = interpolating spline)
    """
    adc_vals = np.array([p[0] for p in calibration_points])
    kev_vals = np.array([p[1] for p in calibration_points])
    
    # Cubic spline interpolation
    from scipy.interpolate import UnivariateSpline
    
    spline = UnivariateSpline(adc_vals, kev_vals, s=smoothing)
    
    # Evaluate residuals
    predicted = spline(adc_vals)
    residuals = kev_vals - predicted
    rms_error = np.sqrt(np.mean(residuals**2))
    
    print(f"Spline calibration:")
    print(f"  Number of knots: {len(spline.get_knots())}")
    print(f"  RMS error: {rms_error:.2f} keV")
    
    return spline, rms_error


cal_spline, rms_spline = spline_calibration(calibration_points_multi)

print(f"\n✓ Spline calibration complete")

## 10. Compare All Calibration Methods

In [ ]:
# Plot comparison
adc_range = np.linspace(500, 3000, 1000)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Calibration curves
ax = axes[0]
ax.plot(adc_range, cal_func_linear_multi(adc_range), 'b-', 
        linewidth=2, label='Linear')
ax.plot(adc_range, cal_func_poly(adc_range), 'r-', 
        linewidth=2, label='Polynomial (deg 2)')
ax.plot(adc_range, cal_spline(adc_range), 'g-', 
        linewidth=2, label='Spline')

# Plot calibration points
adc_points = [p[0] for p in calibration_points_multi]
kev_points = [p[1] for p in calibration_points_multi]
ax.scatter(adc_points, kev_points, s=100, c='black', 
          marker='o', edgecolors='white', linewidth=2, 
          label='Calibration points', zorder=5)

ax.set_xlabel('ADC Channel', fontsize=12, fontweight='bold')
ax.set_ylabel('Energy (keV)', fontsize=12, fontweight='bold')
ax.set_title('Calibration Functions', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Residuals
ax = axes[1]

residuals_linear = np.array(kev_points) - cal_func_linear_multi(adc_points)
residuals_poly = np.array(kev_points) - cal_func_poly(adc_points)
residuals_spline = np.array(kev_points) - cal_spline(adc_points)

x_pos = np.arange(len(kev_points))
width = 0.25

ax.bar(x_pos - width, residuals_linear, width, label='Linear', 
       color='blue', alpha=0.7, edgecolor='black')
ax.bar(x_pos, residuals_poly, width, label='Polynomial', 
       color='red', alpha=0.7, edgecolor='black')
ax.bar(x_pos + width, residuals_spline, width, label='Spline', 
       color='green', alpha=0.7, edgecolor='black')

ax.axhline(0, color='black', linestyle='-', linewidth=1.5)
ax.set_xlabel('Calibration Point', fontsize=12, fontweight='bold')
ax.set_ylabel('Residual (keV)', fontsize=12, fontweight='bold')
ax.set_title('Calibration Residuals', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')
ax.set_xticks(x_pos)
ax.set_xticklabels([f"{int(k)}" for k in kev_points], rotation=45)

plt.tight_layout()
plt.show()

print("✓ Calibration comparison plotted")

## 11. Energy Resolution Calculation

In [ ]:
def calculate_energy_resolution(spectrum_kev, peak_energy_kev, 
                               fit_width_kev=100):
    """
    Calculate energy resolution at a specific peak
    
    Resolution = FWHM / E_peak * 100%
    """
    # Create histogram around peak
    energy_range = (peak_energy_kev - fit_width_kev, 
                   peak_energy_kev + fit_width_kev)
    mask = (spectrum_kev > energy_range[0]) & (spectrum_kev < energy_range[1])
    
    counts, bins = np.histogram(spectrum_kev[mask], bins=100)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    
    # Find peak
    peak_idx = np.argmax(counts)
    peak_pos = bin_centers[peak_idx]
    
    # Fit Gaussian
    def gaussian(x, amp, mu, sigma):
        return amp * np.exp(-0.5 * ((x - mu) / sigma)**2)
    
    try:
        popt, _ = optimize.curve_fit(
            gaussian, bin_centers, counts,
            p0=[counts[peak_idx], peak_pos, 20]
        )
        
        amp, mu, sigma = popt
        fwhm = 2.355 * abs(sigma)
        resolution = (fwhm / mu) * 100
        
        print(f"Energy resolution at {peak_energy_kev} keV:")
        print(f"  Fitted centroid: {mu:.2f} keV")
        print(f"  FWHM: {fwhm:.2f} keV")
        print(f"  Resolution: {resolution:.2f}%")
        
        return resolution, fwhm, mu
    
    except:
        print("Warning: Resolution fit failed")
        return None, None, None


# Calculate resolution for Cs-137 peak
resolution, fwhm, centroid = calculate_energy_resolution(
    energy_kev_calibrated, true_energies[0]
)

print(f"\n✓ Energy resolution calculated")

## Summary and Best Practices

### Key Takeaways

1. **Calibration Source Selection**
   - Use multiple sources spanning your energy range
   - Common choices: Na-22, Cs-137, Co-60
   - For organics: Compton edge often more reliable than photopeak

2. **Method Selection**
   - **Linear**: Fast, adequate for narrow range (±50%)
   - **Polynomial**: Better for wide range, handles non-linearity
   - **Spline**: Most flexible, requires many calibration points

3. **Validation**
   - Check residuals at all calibration points
   - RMS error should be < 1% of energy range
   - Verify with independent source

4. **Quality Metrics**
   - Energy resolution: FWHM/E × 100%
   - Typical organic scintillator: 7-10% at 662 keV
   - NaI(Tl): ~6-7% at 662 keV
   - HPGe: ~0.2% at 662 keV

### Practical Tips

- Recalibrate periodically (temperature effects, aging)
- Store calibration coefficients with data
- Use at least 2 points for linear, 3+ for polynomial
- Check for gain drift using long-lived source
- Account for dead time at high rates

### Next Steps

In Notebook 3, we'll extract advanced timing features from calibrated waveforms for improved PSD discrimination.